<h1>VIT Testing - tamncheese Jason Kahei Tam<h1>


Import

In [1]:
import torch
import torch.nn as nn
import os
import io
from torch.utils.data import DataLoader, Dataset
from transformers import AutoModelForImageClassification, AutoImageProcessor
from PIL import Image
import cv2
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.linear_model import LogisticRegression


path_to_images = "/storage/scratch1/5/jtam30/fungi2024"
files = os.listdir("/storage/scratch1/5/jtam30/fungi2024")
print(files[:2])
path_to_images_val = "/storage/scratch1/5/jtam30/fungi2024"
# path_to_images_val = "/storage/scratch1/5/jtam30/valfungi2024"
# files_val = os.listdir("/storage/scratch1/5/jtam30/valfungi2024")
# print(files_val[:2])

['2883270542-366099.jpg', '2237857759-75860.jpg']


CPU or CUDA


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Prepare Dataset

In [3]:
file_list = []
for (
    root,
    dirs,
    files,
) in os.walk("testing"):
    for file in files:
        file_list.append(file)
file_list

train_df = pd.read_csv(
    "/storage/home/hcoda1/5/jtam30/clef/fungiclef-2025/user/tamncheese/_dataset_trial/metadata_trial.csv"
)

# train_df = pd.read_csv(
#     "/storage/scratch1/5/jtam30/FungiCLEF2023_train_metadata_PRODUCTION.csv"
# )
train_df = train_df.dropna(subset=["species"]).reset_index(drop=True)

n_labels = train_df["species"].nunique()
print(f"Training Dataset has number of labels: {n_labels}")

val_df = pd.read_csv(
    "/storage/home/hcoda1/5/jtam30/clef/fungiclef-2025/user/tamncheese/_dataset_trial/metadata_trial.csv"
)

# val_df = pd.read_csv(
#     "/storage/scratch1/5/jtam30/FungiCLEF2023_val_metadata_PRODUCTION.csv"
# )

val_df = val_df.dropna(subset=["species"]).reset_index(drop=True)

n_labels_val = val_df["species"].nunique()
print(f"Validation Dataset has number of labels: {n_labels_val}")

Training Dataset has number of labels: 1261
Validation Dataset has number of labels: 1261


Define FungiDataset

In [4]:
class FungiDataset(Dataset):
    def __init__(self, df, extractor, transform=None, local_filepath=None):
        self.df = df
        self.transform = transform
        self.extractor = extractor
        self.local_filepath = local_filepath
        self.label2id = {
            label: idx for idx, label in enumerate(sorted(self.df["species"].unique()))
        }  # convert labels to integers

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # label = self.df.iloc[idx]["species"]
        # print(self.local_filepath)
        if self.local_filepath:
            # print("path to image is provided")
            if "val" not in self.local_filepath:
                img_path = os.path.join(
                    self.local_filepath,
                    self.df["image_path"].values[idx].replace("JPG", "jpg"),
                )
            else:
                img_path = os.path.join(
                    self.local_filepath,
                    self.df["image_path"].values[idx],
                )

            # print(img_path)
            species_name = self.df.iloc[idx]["species"]
            label = self.label2id[species_name]
            try:
                # Load Images (OpenCV)
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            except Exception as e:
                print(f"Missing image: {img_path}: {e}")
                image = np.random.uniform(-1, 1, size=(299, 299, 3)).astype(np.float32)
        else:
            print("no path is provided")
            image = Image.open(io.BytesIO(self.df.data.values[idx]))
            image = np.array(image)

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        image = self.extractor(images=image, return_tensors="pt")[
            "pixel_values"
        ].squeeze(0)
        return image, torch.tensor(label, dtype=torch.long)

Configure DINOv2


In [5]:
extractor = AutoImageProcessor.from_pretrained("facebook/dinov2-base")
model = AutoModelForImageClassification.from_pretrained(
    "facebook/dinov2-base", num_labels=n_labels
)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model = model.to(device)
print("Trainable Layers")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")

Trainable Layers
dinov2.embeddings.cls_token is trainable
dinov2.embeddings.mask_token is trainable
dinov2.embeddings.position_embeddings is trainable
dinov2.embeddings.patch_embeddings.projection.weight is trainable
dinov2.embeddings.patch_embeddings.projection.bias is trainable
dinov2.encoder.layer.0.norm1.weight is trainable
dinov2.encoder.layer.0.norm1.bias is trainable
dinov2.encoder.layer.0.attention.attention.query.weight is trainable
dinov2.encoder.layer.0.attention.attention.query.bias is trainable
dinov2.encoder.layer.0.attention.attention.key.weight is trainable
dinov2.encoder.layer.0.attention.attention.key.bias is trainable
dinov2.encoder.layer.0.attention.attention.value.weight is trainable
dinov2.encoder.layer.0.attention.attention.value.bias is trainable
dinov2.encoder.layer.0.attention.output.dense.weight is trainable
dinov2.encoder.layer.0.attention.output.dense.bias is trainable
dinov2.encoder.layer.0.layer_scale1.lambda1 is trainable
dinov2.encoder.layer.0.norm2.wei

Frreze Backbone

In [7]:
for param in model.base_model.parameters():
    param.requires_grad = False

print("Trainable Layers")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name} is trainable")

Trainable Layers
classifier.weight is trainable
classifier.bias is trainable


Load Data


In [21]:
img_dir = path_to_images + "/"
print(f"local_filepath is {img_dir}")

train_dataset = FungiDataset(train_df, extractor, local_filepath=img_dir)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

img_val_dir = path_to_images_val + "/"
val_dataset = FungiDataset(val_df, extractor, local_filepath=img_val_dir)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

local_filepath is /storage/scratch1/5/jtam30/fungi2024/


Ranked List Code

In [22]:
# function in progress
# softmax the logits and extract the top-k predictions

Train the Model


In [23]:
optimizer = torch.optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()), lr=0.00001
)
criterion = nn.CrossEntropyLoss()

epochs = 10
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")

    model.train()
    train_loss = 0
    for images, labels in train_loader:
        # print(images)
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)
    print(f"Training Loss: {avg_train_loss} for epoch {epoch + 1}/{epochs}")

    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validating"):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images).logits
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    print(f"Validation Loss: {avg_val_loss:.4f}, Accuracy: {val_accuracy:.2f}%\n")

Epoch 1/10
Training Loss: 7.695611953735352 for epoch 1/10


Validating:   0%|          | 0/47 [00:00<?, ?it/s]

Validation Loss: 7.5608, Accuracy: 0.10%

Epoch 2/10
Training Loss: 7.501148568822982 for epoch 2/10


Validating:   0%|          | 0/47 [00:00<?, ?it/s]

Validation Loss: 7.3823, Accuracy: 0.30%

Epoch 3/10
Training Loss: 7.33756755260711 for epoch 3/10


...

Validation Loss: 6.5579, Accuracy: 1.99%

Epoch 9/10
Training Loss: 6.519043790533187 for epoch 9/10


Validating:   0%|          | 0/47 [00:00<?, ?it/s]

Validation Loss: 6.4392, Accuracy: 2.20%

Epoch 10/10
Training Loss: 6.407730975049607 for epoch 10/10


Validating:   0%|          | 0/47 [00:00<?, ?it/s]

Validation Loss: 6.3244, Accuracy: 2.50%



Saving the Model

In [24]:
torch.save(
    model.state_dict(),
    "/storage/home/hcoda1/5/jtam30/clef/fungiclef-2025/user/tamncheese/_dataset_trial/model_trial.pth",
)

Extract Embeddings

In [2]:
model = AutoModelForImageClassification.from_pretrained(
    "facebook/dinov2-base", num_labels=n_labels
)
model.eval().to(device)

features = []
labels = []

with torch.no_grad():
    for images, label in train_loader:
        image_list = [image.permute(1, 2, 0).cpu().numpy() for image in images]

        # Extract features for batch
        # batch_features = np.vstack([extract_features(model, img) for img in image_list])
        # features.append(batch_features)
        labels.append(label.numpy())

features = np.vstack(features)
labels = np.hstack(labels)

log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(features, labels)

test_img_path = "/storage/scratch1/5/jtam30/fungi2024/2883270542-366099.jpg"
image = cv2.imread(test_img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

inputs = extractor(images=image, return_tensors="pt")
pixel_values = inputs["pixel_values"].to(device)

with torch.no_grad():
    outputs = model(**inputs)

# Extract the CLS token feature
features = outputs.last_hidden_state[:, 0, :].cpu().numpy()

class_names = {v: k for k, v in train_dataset.label2id.items()}
class_names = [class_names[i] for i in range(len(class_names))]

predicted_label = log_reg.predict(features)[0]
predicted_species = class_names[predicted_label]

print(predicted_species)

NameError: name 'AutoModel' is not defined